1. import necessary libraries

In [ ]:
import argparse
import logging
import os
import random
import numpy as np
import torch
import torch.backends.cudnn as cudnn
from networks.vision_transformer import SwinUnet as ViT_seg
from trainer import trainer_synapse
from config import get_config

2. based on the deafult argument of parser we create the required paths and put the required files in the directories

the most important ones are:
1. data > train and test
2. the train data list
3. and pretrained model

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--root_path', type=str, default='../data/Synapse/train_npz', help='root dir for data')
parser.add_argument('--dataset', type=str, default='Synapse', help='experiment_name')
parser.add_argument('--list_dir', type=str, default='./lists/lists_Synapse', help='list dir')
parser.add_argument('--num_classes', type=int,default=9, help='output channel of network')
parser.add_argument('--output_dir', type=str, help='output dir')                   
parser.add_argument('--max_iterations', type=int, default=30000, help='maximum epoch number to train')
parser.add_argument('--max_epochs', type=int, default=150, help='maximum epoch number to train')
parser.add_argument('--batch_size', type=int, default=24, help='batch_size per gpu')
parser.add_argument('--n_gpu', type=int, default=1, help='total gpu')
parser.add_argument('--deterministic', type=int,  default=1,  help='whether use deterministic training')
parser.add_argument('--base_lr', type=float,  default=0.01,help='segmentation network learning rate')
parser.add_argument('--img_size', type=int,default=224, help='input patch size of network input')
parser.add_argument('--seed', type=int,default=1234, help='random seed')
parser.add_argument('--cfg', type=str, required=True, metavar="FILE", help='path to config file', )
parser.add_argument(  "--opts", help="Modify config options by adding 'KEY VALUE' pairs. ", default=None,nargs='+',)
parser.add_argument('--zip', action='store_true', help='use zipped dataset instead of folder dataset')
parser.add_argument('--resume', help='resume from checkpoint')
parser.add_argument('--accumulation-steps', type=int, help="gradient accumulation steps")
parser.add_argument('--use-checkpoint', action='store_true', help="whether to use gradient checkpointing to save memory")
parser.add_argument('--amp-opt-level', type=str, default='O1', choices=['O0', 'O1', 'O2'],  help='mixed precision opt level, if O0, no amp is used')
parser.add_argument('--tag', help='tag of experiment')
parser.add_argument('--eval', action='store_true', help='Perform evaluation only')
parser.add_argument('--throughput', action='store_true', help='Test throughput only')
parser.add_argument('--cache-mode', type=str, default='part', choices=['no', 'full', 'part'],help='no: no cache, '
                                                                                                   'full: cache all data, '
                                                                                              'part: sharding the dataset into nonoverlapping pieces and only cache one piece')


3. we are working on synapse dataset in this code: so the code is picking the right direcotry where we put our synapse dataset if Synapse is chosen

In [ ]:
args = parser.parse_args()
if args.dataset == "Synapse":           
    args.root_path = os.path.join(args.root_path, "train_npz")

4. there is a configuration file. by using get_config we are getting the parsed arguments and updating the configuration file and creating a new config file to be used later on

In [ ]:
config = get_config(args)

5. here now we are ready to train. before that we create our model to be trained

In [ ]:
if __name__ == "__main__":
    if not args.deterministic:
        cudnn.benchmark = True
        cudnn.deterministic = False
    else:
        cudnn.benchmark = False
        cudnn.deterministic = True

    #preprarion elements that need random number with a seed so random numbers created by them be predictable
    random.seed(args.seed) #random (predictable) number
    np.random.seed(args.seed) #random (predictable) numbpy array
    torch.manual_seed(args.seed) #Sets the seed for generating random numbers. Returns a torch.Generator object.
    torch.cuda.manual_seed(args.seed) #Sets the seed for generating random numbers for the current GPU. It’s safe to call this function if CUDA is not available; in that case, it is silently ignored.

    dataset_name = args.dataset #retrieve the dataset name from args set before and if it is synapse configure a dict for it as follows
    #for the Synapse dataset we have the following dictionary that for dataset=synapse there is a set of configurations
    #if the dataset is something except Synapse then its config is added in the following dict and ( read *)
    dataset_config = {
        'Synapse': {
            'root_path': args.root_path,
            'list_dir': './lists/lists_Synapse',    
            'num_classes': 9,
        },
    }

    if args.batch_size != 24 and args.batch_size % 6 == 0:
        args.base_lr *= args.batch_size / 24
    # * ..... and then values such as sumber of classes would be retrived based on name of dataset from dataset config > here we use synapse dataset so numberof classes
    #is as it is indicated in Synampese dataset_config
    args.num_classes = dataset_config[dataset_name]['num_classes']
    args.root_path = dataset_config[dataset_name]['root_path']
    args.list_dir = dataset_config[dataset_name]['list_dir']

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    #we have now directories and data probabaly ready for our NN layer
    #########################################################################################################################
    #first we create a SwinUNet Layer object as following, this object takes the config, and other parameters as below
    net = ViT_seg(config, img_size=args.img_size, num_classes=args.num_classes).cuda()  # how ? check  SwinUNet in network>vision_transformer
    #loads the pretrained model : how ? check the function load_from : in networks>vision_transformer
    net.load_from(config)
    # now we have a model with pretrained model parameters
    ##########################################################################################################################
    
    trainer = {'Synapse': trainer_synapse,}
    trainer[dataset_name](args, net, args.output_dir) #from trainer import trainer_synapse giving arguments : args, net, args.output_dir